# 语义分割模型使用实例——道路分割

首先导入必要的包

In [ ]:
import sys
sys.path.append('../dataset_preparation')
sys.path.append('../training')
sys.path.append('../inference')
import linestrip_to_mask, split_image_mask_train, split_train_val_test, visualize_mask
import train
import inference, skeleton, road_network_detect_main
import requests
import zipfile
import os
import gdown

## 1.数据准备

运行以下代码下载数据集

In [ ]:
url = 'https://drive.google.com/file/d/1oFHu6KEzEHMrnKwSJ8iye4Ggp1BZ-czD/view?usp=sharing'
output_path = 'road_network_detection.zip'
gdown.download(url, output_path, quiet=False, fuzzy=True)

In [ ]:
! mkdir dataset
! mv road_network_detection.zip dataset
! unzip dataset/road_network_detection.zip

运行linestrip_to_mask程序，可将geojson格式道路标注转化为mask。line_width为道路宽度(米)

example数据集已经将linestrip转为mask，可以跳过这一步

In [ ]:
img_dir = 'dataset/train/images'
geojson_dir = 'dataset/train/labels'
linestrip_to_mask.linestring_to_mask_batch(img_dir, geojson_dir, line_width=4)

运行split_train_val_test程序，可将数据集划分为训练、验证、测试集。验证集测试集比例可以调整。example数据集已经将数据集划分为了train/val，可以跳过这一步

In [ ]:
img_path = 'dataset/train/images'
label_path = 'dataset/train/labels'
split_train_val_test.split_train_val_test(img_path, label_path, val_percentage=0.1, test_percentage=0.1)

运行split_image_mask_train程序，可以将大的tif图片切割成小图，方便后续识别，可以跳过这一步

In [ ]:
# 切割尺寸（单位：米）
split_sizes = [[500, 500], [700, 700], [900, 900]]
images_folder_path = 'dataset/train/images'
masks_folder_path = 'dataset/train/labels'
# 切割后图片尺寸（和后面模型输入尺寸对应）
input_img_size = 1024
split_image_mask_train.split_images_segment_v1(images_folder_path, split_sizes, masks_folder_path, input_img_size)

## 2. 模型训练

运行train.py启动训练。训练可调参数如下。base_dir里面需要有train/val两个文件夹。

In [ ]:
base_dir = 'dataset/'
# 模型架构可以从这几个里面选
# ['unet', 'unetplusplus', 'manet', 'linknet', 'fpn', 'pspnet', 'deeplabv3', 'deeplabv3plus', 'pan']
# https://github.com/qubvel/segmentation_models.pytorch
architecture = 'unet'

encoder = 'resnet34'
device = [0, 1, 2, 3]
save_model_every_n_epochs = 25
max_epochs = 200
checkpoint_callback_dir = os.path.join(base_dir, 'model_checkpoints/')
img_size = 640
batch_size = 8
initial_learning_rate = 1e-3
patience = 20
log_dir = os.path.join(base_dir, 'tb_logs')
# tensorboard --logdir=log_dir
# 添加本地映射命令：ssh -L 6006:localhost:6006 zkxq@192.168.50.6
name = 'access_road'

seg_data_module = train.setup_data_module(base_dir, img_size, batch_size)
model, trainer = train.setup_model_and_trainer(architecture, encoder, save_model_every_n_epochs, max_epochs,
                                         checkpoint_callback_dir, initial_learning_rate, patience,
                                         log_dir, name, device)
tuner = train.Tuner(trainer)
# Auto-scale batch size by growing it exponentially (default)
# tuner.scale_batch_size(model, mode="power")
tuner.lr_find(model, datamodule=seg_data_module)
trainer.fit(model, datamodule=seg_data_module)
valid_metrics = trainer.validate(model, seg_data_module, verbose=False)
test_metrics = trainer.test(model, seg_data_module, verbose=False)

pprint(valid_metrics)
pprint(test_metrics)

## 2. 模型预测

运行road_network_detect_main.py预测模型。在config/road_network.yaml中指明模型路径以及切割图片大小。程序最终会在out_dir里生成每张图片对应的geojson文件

In [ ]:
config_dict = road_network_detect_main.get_config()
image_path = "/home/zkxq/project/hjt/smp/datasets/Wind_Turbine/test/test_img"
out_dir = "/home/zkxq/project/hjt/smp/datasets/Wind_Turbine/out"
   
# config_dict['start_time'] = start_time
mid_path = os.path.join(image_path, 'mid_result')
os.makedirs(mid_path, exist_ok=True)
config_dict['mid_path'] = mid_path
config_dict['image_path'] = image_path
config_dict['image_paths'] = road_network_detect_main.change_img_path(image_path)
config_dict['out_dir'] = out_dir

road_network_detect_main.model_predict(config_dict)
road_network_detect_main.after_handle(config_dict)
print(f'结果已存储在：{out_dir}')